In [44]:
import os
import shutil
import pandas as pd

# Path ke folder dataset Anda
base_path = r'D:\Tugas Unpam\SEMESTER 5\MACHINE LEARNING\TUGAS YOLO\batu-gunting-kertas-2'

def rapikan_dataset_final(subset):
    path_subset = os.path.join(base_path, subset)
    csv_path = os.path.join(path_subset, "_classes.csv")
    
    if not os.path.exists(csv_path):
        print(f"⚠️ Tidak ada CSV di {subset}, mencoba menebak folder...")
        return

    # Baca CSV untuk tahu gambar mana masuk kelas mana
    df = pd.read_csv(csv_path)
    
    # Ambil nama kolom selain 'filename' (biasanya: paper, rock, scissors)
    classes = [col for col in df.columns if col != 'filename']
    
    for _, row in df.iterrows():
        img_name = row['filename'].strip()
        src = os.path.join(path_subset, img_name)
        
        if os.path.exists(src):
            # Cari kolom mana yang nilainya 1 (misal kolom 'rock' bernilai 1)
            for cls in classes:
                if row[cls] == 1:
                    dest_folder = os.path.join(path_subset, cls)
                    os.makedirs(dest_folder, exist_ok=True)
                    shutil.move(src, os.path.join(dest_folder, img_name))
                    break
    
    # Hapus file cache lama agar YOLO tidak membaca data lama yang error
    cache_path = os.path.join(base_path, f"{subset}.cache")
    if os.path.exists(cache_path):
        os.remove(cache_path)
        
    print(f"✅ Selesai merapikan {subset}. Sekarang folder sudah terbagi sesuai kelas.")

# Jalankan untuk train, valid, dan test
for s in ['train', 'valid', 'test']:
    rapikan_dataset_final(s)

✅ Selesai merapikan train. Sekarang folder sudah terbagi sesuai kelas.
✅ Selesai merapikan valid. Sekarang folder sudah terbagi sesuai kelas.
✅ Selesai merapikan test. Sekarang folder sudah terbagi sesuai kelas.


In [45]:
from ultralytics import YOLO

model = YOLO('yolov8n-cls.pt')

model.train(
    data=r'D:\Tugas Unpam\SEMESTER 5\MACHINE LEARNING\TUGAS YOLO\batu-gunting-kertas-2',
    epochs=15,
    imgsz=224,
    batch=4,
    workers=0,
    name='rps_final_success'
)

Ultralytics 8.3.240  Python-3.13.9 torch-2.9.1+cpu CPU (AMD Ryzen 3 7320U with Radeon Graphics)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=D:\Tugas Unpam\SEMESTER 5\MACHINE LEARNING\TUGAS YOLO\batu-gunting-kertas-2, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=15, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=rps_final_success4, nbs=64, nms=False, opset=None, optimize=False, optimizer

ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000027E75A76AC0>
curves: []
curves_results: []
fitness: 1.0
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 1.0, 'metrics/accuracy_top5': 1.0, 'fitness': 1.0}
save_dir: WindowsPath('C:/runs/classify/rps_final_success4')
speed: {'preprocess': 0.0062850000176695175, 'inference': 9.994654999991326, 'loss': 0.00024000000848900527, 'postprocess': 0.0007250000180647476}
task: 'classify'
top1: 1.0
top5: 1.0

In [46]:
from ultralytics import YOLO
import os

# 1. Load model yang baru saja selesai di-train
model_path = r'C:\runs\classify\rps_final_success\weights\best.pt'
model_rps = YOLO(model_path)

# 2. Ambil satu gambar dari folder test untuk uji coba
# Kita ambil gambar pertama dari salah satu folder (paper/rock/scissors)
test_dir = r'D:\Tugas Unpam\SEMESTER 5\MACHINE LEARNING\TUGAS YOLO\batu-gunting-kertas-2\test'
subfolders = [f for f in os.listdir(test_dir) if os.path.isdir(os.path.join(test_dir, f))]
sample_img_path = os.path.join(test_dir, subfolders[0], os.listdir(os.path.join(test_dir, subfolders[0]))[0])

# 3. Prediksi
results = model_rps.predict(source=sample_img_path, save=True)

# 4. Tampilkan hasil
for result in results:
    # Mengambil indeks kelas terbaik
    top1_idx = result.probs.top1
    
    # Mengambil nama kelas berdasarkan indeks tersebut
    top1_name = result.names[top1_idx]
    
    # Mengambil nilai kepercayaan (confidence)
    top1_conf = result.probs.top1conf.item()

    print(f"\n--- HASIL PREDIKSI ---")
    print(f"Gambar: {sample_img_path}")
    print(f"Prediksi: {top1_name}") 
    print(f"Keyakinan (Confidence): {top1_conf:.2%}")


image 1/1 D:\Tugas Unpam\SEMESTER 5\MACHINE LEARNING\TUGAS YOLO\batu-gunting-kertas-2\test\ batu\1e1VCnCEApnFh9Oo_png.rf.613906a0dde559e787965c9aa6c1d8b0.jpg: 224x224  batu 1.00,  gunting 0.00,  kertas 0.00, scissors 0.00, rock 0.00, 20.3ms
Speed: 9.9ms preprocess, 20.3ms inference, 0.1ms postprocess per image at shape (1, 3, 224, 224)
Results saved to C:\runs\classify\predict3

--- HASIL PREDIKSI ---
Gambar: D:\Tugas Unpam\SEMESTER 5\MACHINE LEARNING\TUGAS YOLO\batu-gunting-kertas-2\test\ batu\1e1VCnCEApnFh9Oo_png.rf.613906a0dde559e787965c9aa6c1d8b0.jpg
Prediksi:  batu
Keyakinan (Confidence): 99.59%
